In [ ]:
import iniciarBD as mi, informacionAdicional as inf
from iniciarBD import meses, web, URL

import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from bs4 import BeautifulSoup
import re
from datetime import date, datetime as dt, timedelta 
import pandas as pd
import os

### WEBSCRAPING
https://www.culturaydeporte.gob.es/cultura/areas/cine/datos/taquilla-espectadores.html

In [ ]:
def Domingo(anho, mes, dia):
  """
  si la fecha dada no es domingo devuelve la fecha del siguiente domingo a la fecha dada
  
  Parameters: anho, mes, dia, integer para componer la fecha
  Returns:  fecha, date
  """
  fecha = date(anho, mes, dia)
  fecha = ( fecha + timedelta(days = 6-fecha.weekday())) if (fecha.weekday() != 6) else fecha
  
  return fecha

In [ ]:
def nueva_info (semana ):
  """
  comprueba si ya hay archivos procesados para una semana dada
  
  Parameters: semana, cadena formato yyyyww
  Returns: True si no hay archivos en la base de datos para esa semana 
  """
  archivos = mi.leer_tabla('taquilla_archivos', f'semana = {semana} ;')

  return archivos.empty

In [ ]:
def ultimos_archivos_taquilla():
  """
  obtiene mediante webscrapping la lista de los últimos archivos disponibles en la página web del ministerio de cultura y deporte donde se recogen los datos de taquilla
  
  Returns: dataframe (periodicidad | fecha | semana | tipo | archivo | url)
  """    
  # evitar mostrar avisos
  requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

  archivos  = []

  page = requests.get(URL, verify=False)
  soup = BeautifulSoup(page.content, "html.parser")

  #sección última información semanal
  tablas = soup.find_all("div", class_="info")
  # patron para buscar archivos con última información  (tipo) (dia) (mes)
  regExp = r"^(Top|Cine|Acumulado).* (\d{,2}) ([\w ]+)$"
  
  for t in tablas :
      referencia = t.find("a").text.strip()   
      url = web + t.find("a").attrs['href']     
      result = re.search(regExp, referencia)
      mes = result[3].split()[-1]
      fecha = Domingo(dt.today().year, int(meses.get(mes)), int(result[2]) )
      semana = str(fecha.isocalendar()[0]) + str(fecha.isocalendar()[1]).zfill(2) 
      if nueva_info(semana):     
        doc = {'periodicidad': "semanal",
               'fecha': fecha,
               'semana': semana,
               'tipo': result[1].lower(),
               'archivo': os.path.basename(url),
               'url': url
               }
        archivos.append(doc)

  # sección Histórico  
  tabla = soup.find("div", class_="cblq fondo")
  #patron para buscar archivos anuales
  regExp = r"^.* (\d{4}).*"
  # solo el primer archivo de la lista
  nombre = tabla.find("a").text.strip()
  url = web + tabla.find("a").attrs['href']
  result = re.search(regExp, nombre)
  semana = result[1] + date(int(result[1]), 12, 28).strftime("%V")
  # si hay nuevo fichero anual hay que procesarlo
  if nueva_info(semana): 
    doc = {'periodicidad': "anual",
           'fecha': date(int(result[1]), 12, 31),
           'semana': semana,
           'tipo': "acumulado",
           'archivo': os.path.basename(url),
           'url': url        
           }
    archivos.append(doc)

  if len(archivos)==0:
    df_archivos = pd.DataFrame(columns=['periodicidad','fecha','semana','tipo','archivo','url'])
  else:
    df_archivos = pd.DataFrame(archivos).sort_values(by = [  'periodicidad', 'semana'], ascending = [True, True])
    
  return df_archivos

In [12]:
nuevos_archivos = ultimos_archivos_taquilla()
nuevos_archivos

,periodicidad,fecha,semana,tipo,archivo,url
0,semanal,2022-09-11,202236,top,top-25-9-11-septiembre-2022.pdf,https://www.culturaydeporte.gob.es/dam/jcr:4b9...
1,semanal,2022-09-11,202236,cine,topespanol-9-11-septiembre-2022.pdf,https://www.culturaydeporte.gob.es/dam/jcr:ca7...
2,semanal,2022-09-11,202236,acumulado,acumulado-11-septiembre-2022.pdf,https://www.culturaydeporte.gob.es/dam/jcr:a55...


### DESCARGAR

In [13]:
mi.descargar_archivos(nuevos_archivos) 

### PROCESAR 

In [ ]:
def siguiente_peli ():
  """
  Returns: int, el siguiente número al último identificador de película 
  """
  db = mi.conectar_bd()
  peli = pd.read_sql ("select MAX(id) as id from peliculas;", db)
  mi.desconectar_bd(db)

  return peli.id.iat[0]+1

In [ ]:
def nuevas_peliculas(da, ds):
  """
  Parameters: da, dataframe con dato acumulado semanal
              ds, dataframe con dato semanal 
  Returns: pelis, datframe con películas estrenadas la última semana
  """
  pelis = ds[ds.SEM_ESTRENO==1][['TITULO', 'TIT_ORIGINAL','DISTRIBUIDORA','FECHA_INFO']]
  pelis = pd.merge(pelis, da[["TITULO", "TIT_ORIGINAL", "DISTRIBUIDORA", "FECHA_ESTRENO"]], how="left", on =["TITULO", "TIT_ORIGINAL", "DISTRIBUIDORA"])
  # si la película no tiene fecha de estreno poner la del viernes anterior a la fecha de la información
  pelis.loc[pelis.FECHA_ESTRENO.isnull(), 'FECHA_ESTRENO'] = pelis.loc[pelis.FECHA_ESTRENO.isnull(),['FECHA_INFO'] ].apply(lambda row: (row.FECHA_INFO - timedelta(days = 2)), axis=1)
  pelis.drop(axis=1, columns='FECHA_INFO', inplace=True) 
  pelis['FECHA_ESTRENO'] = pd.to_datetime(pelis.FECHA_ESTRENO).dt.date
  pelis['TMDB_ID']=0
  pelis.index=list(range(siguiente_peli(),siguiente_peli()+len(pelis)))
  pelis.reset_index(inplace=True)
  pelis.rename (columns={'index':'id', 'FECHA_ESTRENO':'FECHA'}, inplace=True)

  return pelis 

In [ ]:
def ultimos_datos(archivos):
  """
  procesa los últimos archivos publicados
  
  Parameters: archivos, dataframe con la lista de archivos
  """    
  log=[] # inicializar lista de errores

  dato_anual = mi.obtener_dato_anual(archivos[archivos['tipo']=="acumulado"], log)

  df_arch_sem = archivos[archivos['tipo']!="acumulado"].pivot_table(index=['semana', 'fecha'], columns='tipo',
                 values='archivo', aggfunc='first').reset_index()
  dato_semanal = mi.obtener_dato_semanal(df_arch_sem, log)

  df_log = pd.DataFrame(log, columns = ['semana','tipo','archivo', 'error' ])
  archivos = pd.merge(archivos, df_log, how="left", on =["semana", "tipo", "archivo"])

  peliculas = nuevas_peliculas(dato_anual, dato_semanal)
    
  inf.buscar_TMDB(peliculas)

  db = mi.conectar_bd()
  
  mi.guardar_tabla(archivos, 'taquilla_archivos',  db, 'append')  
  mi.guardar_tabla(dato_anual, 'taquilla_anual', db, 'append')
  mi.guardar_tabla(dato_semanal, 'taquilla_semanal', db, 'append')
  mi.guardar_tabla(peliculas, 'peliculas', db, 'append' )

  mi.desconectar_bd(db)  

  # borrar datos acumulados de la semana anterior
  mi.borrar_datos('taquilla_anual', filtro=f'semana_info = {int(archivos.semana.iat[0])-1}')

  inf.obtener_detalles(peliculas, 'append')

  ## devolucion para poder visualizar un resultdo en notebook <<<<<<<<<<<<<
  return peliculas 
    

In [14]:
peliculas = ultimos_datos(nuevos_archivos) if not nuevos_archivos.empty else print('nada')
peliculas

,id,TITULO,TIT_ORIGINAL,DISTRIBUIDORA,FECHA,TMDB_ID
0,2392,viaje al paraiso,ticket to paradise,universal,2022-09-09,800939
1,2393,"huerfana: primer asesinato, la",orphan: first kill,diamond films,2022-09-09,760161
2,2394,"colegio de los animales magicos, el","schule der magischen tiere, die",flins y piniculas,2022-09-09,793937
3,2395,"acusado, el","choses humaines, les",karma,2022-09-09,756603
4,2396,brahmastra part 1: shiva,brahmastra part 1: shiva,pavan,2022-09-09,496331
5,2397,jaula,jaula,sony,2022-09-09,960875
6,2398,duo,duo,atalante,2022-09-09,-1
7,2399,mi vacio y yo,mi vacio y yo,filmin,2022-09-09,924125


In [15]:
print(f' where tmdb_id in ({",".join(peliculas.TMDB_ID.apply(str).tolist())})')

 where tmdb_id in (800939,760161,793937,756603,496331,960875,-1,924125)
